# Analysis operations

### Import Packages

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import fiona
import h5py
import re
from statistics import mean

# Andy's imports:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import glob
import sys
import pyproj
import scipy.interpolate as scinterp

%matplotlib inline

### Visualize data with MODIS true color imagery 

* Overlay ICESat-2 tracks on MODIS Corrected Reflectance as a 3-D plot or using cartopy (optional)
    * Overlay photon/height/freeboard on WorldView imagery
    * Use highest resolution possible for WorldView
    * Example of visualization of data – not really useful for analysis but “pretty” image with qualitative information
    * Higher ATL07 heights along fast ice, lower heights over thin ice and then increasing height.
    * ATL03 should show fewer photon returns over open water and thin ice


* Plot IS-2 track on MOD29 IST data
    * Same as for WorldView but using IST
    * May need to composite two or more tiles for coverage
    * May need/want a land mask


In [2]:
# example url
url = 'https://gibs.earthdata.nasa.gov/twms/epsg4326/best/twms.cgi?request=GetMap&layers=MODIS_Terra_CorrectedReflectance_TrueColor&srs=EPSG:4326&format=image/jpeg&styles=&time=2012-07-09&width=512&height=512&bbox=-18,27,-13.5,31.5'

### Data filtering by quality flag



### Extract MODIS values that overlap ICESat-2 tracks

* Match up MODIS and ICESat-2 values – depending on IS-2 segment length vs. MODIS footprint, may simply due nearest neighbor, or may want to average (bin) on or the other
* Use MODIS footprint as the baseline and match/bin IS-2 to each MODIS observation


Based on Andy's ICESat-2 hackweek tutorial to use Xarray to extract gridded data along ICESat-2 tracks:


https://github.com/ICESAT-2HackWeek/xarray_open_tutorial/blob/master/xarray_examples.ipynb

### Plot MODIS IST (MOD29) and ICESat-2 height/freeboard along tracks (X-axis = position, Y-axis = IST and height)

* This should show cold IST over the fast ice with fairly flat and higher ICESat-2 heights
* Warm IST will be seen in open water, thin ice just off the fast ice boundary
    * IST will decrease along transect
    * IS-2 should show low heights, but probably low quality (at least for ATL10, if any are available)

### Plot MODIS IST vs. ICESat-2: scatterplot (X-axis = IST, Y-axis = IS-2 height)

* Should show a pretty clear relationship
    * Over fast ice: IST should be close to constant and height will also be fairly constant
    * Over drift ice, where it goes from thin ice toward thicker ice, should see height increasing as IST decreases
    * If we use IST = 271.2 K for open water and then find the IST at the first valid ATL10 value, we have the IST value at that freeboard. Then can use that relationship to get thin ice freeboard where there is ATL07 data, but not ATL10


### Add AMSR2 (AU_SI6) 89V-H difference to MODIS IST and ICESat-2 plots

* Difference for open water/thin ice large (~40-50 K), for thicker ice near zero (<10 K)
* Or calculate the polarization ratio (PR) = (89V – 89H)/(89V + 89H), which is commonly used in the PM sea ice community
* In theory can get concentration from PR (or 89 difference), but don’t really need to do that here.


For each beam group within each file, filter /sea_ice_segments/heights/height_segment_height based on:

/sea_ice_segments/heights/height_segment_quality

### A couple other possibilities:
* Compare cross-over points between tracks 1298 and 1305. 
    * This is over drift ice, but might be able to correlate between them
    * Using all strong beams, there are 9 cross-over points, so there is 
* Compare 1298 repeat track from 12/22/2018 and 3/23/2019
    * For fast ice, should see an increase in height from Dec to Mar as snow depth and ice thickness increases
        * From a quick look at OA, I’m not really seeing this though
        * Not much from ATL10
        * Lots of data in ATL07, but don’t really see a connection – in fact Dec looks lower than Mar; this could be due to corrections/quality issues